# Function Calls in Lettuce

Previously, we did not handle function calls in our fledgling Lettuce interpreter.  Now we will add support for function calls.  Before we do so, let us recall the full grammar.



$$\begin{array}{rcll}
\mathbf{Program} & \rightarrow & TopLevel(\mathbf{Expr}) \\[5pt]
\mathbf{Expr} & \rightarrow & Const(\mathbf{Number}) \\
 & | & True \\
 & | & False \\
 & | & Ident(\mathbf{Identifier}) \\
 & | & Plus(\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Minus(\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Mult (\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Div (\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Log (\mathbf{Expr}) \\
 & | & Exp (\mathbf{Expr}) \\
 & | & Sine (\mathbf{Expr}) \\
 & | & Cosine (\mathbf{Expr}) \\
 & | & Geq (\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Eq (\mathbf{Expr}, \mathbf{Expr}) \\
 & | & And ( \mathbf{Expr}, \mathbf{Expr} ) \\
 & | & Or ( \mathbf{Expr}, \mathbf{Expr} ) \\
 & | & Not ( \mathbf{Expr}) \\
 & | & IfThenElse(\mathbf{Expr}, \mathbf{Expr}, \mathbf{Expr}) & \text{if (expr) then expr else expr} \\
 & | & Let( \mathbf{Identifier}, \mathbf{Expr}, \mathbf{Expr}) & \text{let identifier = expr in expr} \\
 & | & FunDef( \mathbf{Identifier}, \mathbf{Expr}) & \text{function (identifier-formal-parameter) expr } \\ 
 & | & FunCall(\mathbf{Expr}, \mathbf{Expr}) & \text{function call - expr(expr)} \\[5pt]
\end{array}$$


Here is the scala definition again.

In [1]:
sealed trait Program
sealed trait Expr

case class TopLevel(e: Expr) extends Program

case class Const(v: Double) extends Expr // Expr -> Const(v)
case object True extends Expr // Expr -> True
case object False extends Expr // Expr -> False
case class Ident(s: String) extends Expr // Expr -> Ident(s)

// Arithmetic Expressions
case class Plus(e1: Expr, e2: Expr) extends Expr // Expr -> Plus(Expr, Expr)
case class Minus(e1: Expr, e2: Expr) extends Expr // Expr -> Minus(Expr, Expr)
case class Mult(e1: Expr, e2: Expr) extends Expr // Expr -> Mult (Expr, Expr)
case class Div(e1: Expr, e2: Expr) extends Expr // Expr -> Mult(Expr, Expr)
case class Log(e: Expr) extends Expr 
case class Exp(e: Expr) extends Expr
case class Sine(e: Expr) extends Expr
case class Cosine(e: Expr) extends Expr

// Boolean Expressions
case class Geq(e1: Expr, e2:Expr) extends Expr
case class Eq(e1: Expr, e2: Expr) extends Expr
case class And(e1: Expr, e2: Expr) extends Expr
case class Or(e1: Expr, e2: Expr) extends Expr
case class Not(e: Expr) extends Expr

//If then else
case class IfThenElse(e: Expr, eIf: Expr, eElse: Expr) extends Expr

//Let bindings
case class Let(s: String, defExpr: Expr, bodyExpr: Expr) extends Expr

//Function definition
case class FunDef(param: String, bodyExpr: Expr) extends Expr

// Function call
case class FunCall(funCalled: Expr, argExpr: Expr) extends Expr

defined trait Program
defined trait Expr
defined class TopLevel
defined class Const
defined object True
defined object False
defined class Ident
defined class Plus
defined class Minus
defined class Mult
defined class Div
defined class Log
defined class Exp
defined class Sine
defined class Cosine
defined class Geq
defined class Eq
defined class And
defined class Or
defined class Not
defined class IfThenElse
defined class Let
defined class FunDef
defined class FunCall

### Example 1

~~~
let square = function(x) 
           x * x in 
    square(10) 
~~~

This program evaluates to 100

In [2]:
val p1 = TopLevel( 
    Let("square",                                // let square = 
         FunDef("x", Mult(Ident("x"), Ident("x"))),  //    function (x) x * x
         FunCall(Ident("square"), Const(10)) //     in  square(10)
       )
)


p1: TopLevel = TopLevel(Let(square,FunDef(x,Mult(Ident(x),Ident(x))),FunCall(Ident(square),Const(10.0))))

### Example 2
~~~
let x = 10 in 
let y = 15 in 
let sq1 = function (x) 
            function (y) 
               x + y * y
           in 
     sq1(x)(y)
~~~
          
__Question:__ Map the different usages of x, y in the code above to the appropriate definitions?

This program evaluates to $235$.


In [3]:
val x = Ident("x")
val y = Ident("y")
val fdef_inner = FunDef("y", Plus(x, Mult(y, y)))
val fdef_outer = FunDef("x", fdef_inner)
val call_expr = FunCall(FunCall(Ident("sq1"), x), y)
val sq1_call = Let("sq1", fdef_outer, call_expr)
val lety = Let("y", Const(15), sq1_call)
val letx = Let("x", Const(10), lety)
val p2 = TopLevel(letx)

x: Ident = Ident("x")
y: Ident = Ident("y")
fdef_inner: FunDef = FunDef(y,Plus(Ident(x),Mult(Ident(y),Ident(y))))
fdef_outer: FunDef = FunDef(x,FunDef(y,Plus(Ident(x),Mult(Ident(y),Ident(y)))))
call_expr: FunCall = FunCall(FunCall(Ident(sq1),Ident(x)),Ident(y))
sq1_call: Let = Let(sq1,FunDef(x,FunDef(y,Plus(Ident(x),Mult(Ident(y),Ident(y))))),FunCall(FunCall(Ident(sq1),Ident(x)),Ident(y)))
lety: Let = Let(y,Const(15.0),Let(sq1,FunDef(x,FunDef(y,Plus(Ident(x),Mult(Ident(y),Ident(y))))),FunCall(FunCall(Ident(sq1),Ident(x)),Ident(y))))
letx: Let = Let(x,Const(10.0),Let(y,Const(15.0),Let(sq1,FunDef(x,FunDef(y,Plus(Ident(x),Mult(Ident(y),Ident(y))))),FunCall(FunCall(Ident(sq1),Ident(x)),Ident(y)))))
p2: TopLevel = TopLevel(Let(x,Const(10.0),Let(y,Const(15.0),Let(sq1,FunDef(x,FunDef(y,Plus(Ident(x),Mult(Ident(y),Ident(y))))),FunCall(FunCall(Ident(sq1),Ident(x)),Ident(y))))))

### Example 3

~~~
let h = function(z) 
         log(z) 
     in 
let g = function(y) 
         y/2.0 + h(y * 1.5)
      in 
let f = function (x) 
         1.0/x + g(x)
in 
    f(3.1415)
~~~

This should evaluate to 3.4392347...

In [4]:
val x = Ident("x")
val y = Ident("y")
val z = Ident("z")

val fDef = FunDef("x", Plus(Div(Const(1.0), x), FunCall(Ident("g"), x)) )
val gDef = FunDef("y", Plus(Div(y, Const(2.0)), FunCall(Ident("h"), Mult(y, Const(1.5)))))
val hDef = FunDef("z", Log(z))

val letf = Let("f", fDef, FunCall(Ident("f"), Const(3.1415)))
val letg = Let("g", gDef, letf)
val leth = Let("h", hDef, letg)

val p3 = TopLevel(leth)

x: Ident = Ident("x")
y: Ident = Ident("y")
z: Ident = Ident("z")
fDef: FunDef = FunDef(x,Plus(Div(Const(1.0),Ident(x)),FunCall(Ident(g),Ident(x))))
gDef: FunDef = FunDef(y,Plus(Div(Ident(y),Const(2.0)),FunCall(Ident(h),Mult(Ident(y),Const(1.5)))))
hDef: FunDef = FunDef(z,Log(Ident(z)))
letf: Let = Let(f,FunDef(x,Plus(Div(Const(1.0),Ident(x)),FunCall(Ident(g),Ident(x)))),FunCall(Ident(f),Const(3.1415)))
letg: Let = Let(g,FunDef(y,Plus(Div(Ident(y),Const(2.0)),FunCall(Ident(h),Mult(Ident(y),Const(1.5))))),Let(f,FunDef(x,Plus(Div(Const(1.0),Ident(x)),FunCall(Ident(g),Ident(x)))),FunCall(Ident(f),Const(3.1415))))
leth: Let = Let(h,FunDef(z,Log(Ident(z))),Let(g,FunDef(y,Plus(Div(Ident(y),Const(2.0)),FunCall(Ident(h),Mult(Ident(y),Const(1.5))))),Let(f,FunDef(x,Plus(Div(Const(1.0),Ident(x)),FunCall(Ident(g),Ident(x)))),FunCall(Ident(f),Const(3.1415)))))
p3: TopLevel = TopLevel(Let(h,FunDef(z,Log(Ident(z))),Let(g,FunDef(y,Plus(Div(Ident(y),Const(2.0)),FunCall(Ident(h),Mult(Ident(y),Const(1.5)

### Example 4 (Bad)

We can ask what happens if we define a recursive function like so:

~~~
let f = function (x) 
         if (0 >= x) 
             1
         else
             (x - 1)* f(x - 1 )
     in 
     f(10)
~~~

We can predict: note that the let expression 

~~~
let f = exprA in expr B
~~~

we note that `f` is __not__ in scope for `exprA`. Therefore, when we make a recursive call `f(x-1)`, we will
get an error, as we will see.

<font color="red"> We will modify this situation to allow recursive definitions coming next. But for now,
this sort of a call will lead to an error.</font>


In [5]:
val x = Ident("x")
val compX = Geq(Const(0), x)
val recExpr = Mult(Minus(x, Const(1.0)), FunCall(Ident("f"), Minus(x, Const(1.0))))
val f_defn = FunDef("x", IfThenElse(compX, Const(1.0), recExpr))
val letf = Let("f", f_defn, FunCall(Ident("f"), Const(10.0)))
val p4 = TopLevel(letf)

x: Ident = Ident("x")
compX: Geq = Geq(Const(0.0),Ident(x))
recExpr: Mult = Mult(Minus(Ident(x),Const(1.0)),FunCall(Ident(f),Minus(Ident(x),Const(1.0))))
f_defn: FunDef = FunDef(x,IfThenElse(Geq(Const(0.0),Ident(x)),Const(1.0),Mult(Minus(Ident(x),Const(1.0)),FunCall(Ident(f),Minus(Ident(x),Const(1.0))))))
letf: Let = Let(f,FunDef(x,IfThenElse(Geq(Const(0.0),Ident(x)),Const(1.0),Mult(Minus(Ident(x),Const(1.0)),FunCall(Ident(f),Minus(Ident(x),Const(1.0)))))),FunCall(Ident(f),Const(10.0)))
p4: TopLevel = TopLevel(Let(f,FunDef(x,IfThenElse(Geq(Const(0.0),Ident(x)),Const(1.0),Mult(Minus(Ident(x),Const(1.0)),FunCall(Ident(f),Minus(Ident(x),Const(1.0)))))),FunCall(Ident(f),Const(10.0))))

## Scoping for Function Calls: Static vs. Dynamic

It is important to understand how function calls capture scopes by considering a few examples both in lettuce and scala. 

### Example 1
~~~
let x = 10 in 
   let f = function(y) y * x in 
      let x = 15 in 
          f(10)
~~~

Or equivalently in scala

~~~
val x = 10
val f = (y: Int) => (y * x) 
{
   val x = 15 
      f(10)
}

~~~

In both cases, our code has a function `f` that multiplies its parameter `y` by `x`. But precisely which of the
`x` should the function use?

  - `let x = 10` in the first line that is in scope when the function was first defined?, or 
  - `let x = 15` in the third line that is in scope when the function is actually being called?


What would the Scala compiler do?

In [3]:
val x = 10 
val f = (y: Int) => (y * x) 
        {
           val x = 15 
           println("f(10) = " + f(10))
        }


f(10) = 100


x: Int = 10
f: Int => Int = ammonite.$sess.cmd2$Helper$$Lambda$2169/512607238@21c3e734

We can conclude that scala uses the definition of `x` that was in scope __when the function was defined__.
This kind of scoping is called __static scoping__. As opposed to dynamic scoping, in which case the value of `x` used is what is in scope when the function is called.

### Why Static Scoping?

Static scoping enables many useful programming idioms. We mention a few.

#### (A) Partial application of functions

Static scoping allows us to define functions some of whose arguments are already bound at definition time.

In [11]:
// matchAgainstList takes in a reference_lst and returns a function
// that itself takes in a list
def matchAgainstList(reference_lst: List[Int]) (lstB: List[Int]) = {
    def belongsToRefList(x: Int) = {
        reference_lst.contains(x)
    }
    lstB.exists(belongsToRefList)
}

val refList = List(1,2, 3, 4, 5, 6, 7, 8, 9, 10)
val boundMatchFun: (List[Int] => Boolean) = matchAgainstList (refList)(_)
{
    val refList = List(11, 12, 13, 14)
    boundMatchFun(List(5,6,7, 8))
}


defined function matchAgainstList
refList: List[Int] = List(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)
boundMatchFun: List[Int] => Boolean = ammonite.$sess.cmd10$Helper$$Lambda$2276/1890012431@1b9d482e
res10_3: Boolean = true

#### (B) Callbacks

In the same vein as before, we can define callbacks to handle events using partially applied function to bind extra information into the callback function.

In [19]:
abstract class Event
case class KeyPress(keyID: Int) extends Event
case class MouseClick(x: Int, y: Int) extends Event
case object PrintStatistics extends Event

def keyPressEventHandlerFactory(): (Event => Unit) = {
    // Make a callback function and return it.
    var keysPressedSoFar: Set[Int] = Set() // Mutable state
    def keyPressEventCallBack(event: Event): Unit = {
        event match {
            case KeyPress(keyID) => { 
                keysPressedSoFar = keysPressedSoFar ++ Set(keyID)
            }
            case PrintStatistics => {
                println("Keys Pressed so far are: " + keysPressedSoFar)
            }
            case _ => ()
        }
    }
    keyPressEventCallBack
}


def handleEventStream() = {
    val eventsToTest = List[Event](KeyPress(1), KeyPress(2), MouseClick(10,20), KeyPress(10), KeyPress(-1), PrintStatistics)
    val keyPressEventCallBack = keyPressEventHandlerFactory()
    eventsToTest.foreach (keyPressEventCallBack)
}


handleEventStream()

Keys Pressed so far are: Set(1, 2, -1, 10)


defined class Event
defined class KeyPress
defined class MouseClick
defined object PrintStatistics
defined function keyPressEventHandlerFactory
defined function handleEventStream

### Implementing Dynamic Scoping

Turns out implementing dynamic scoping is conceptually easy. We could just ``inline'' the body of the function and substitute for the callee argument. 

~~~
let x = 10 in 
   let f = function(y) y * x in 
      let x = 15 in 
          f(10)
~~~


Everywhere `f` is called, simply replace it by `y * x` and substitute the argument of the call in place of `y`.
If we did that, this would be the result and you get dynamic scoping.

~~~
let x = 10 in 
    let f = function (y) y * x in 
       let x = 15 in 
           (10 * x)
~~~

#### Example 2

~~~
  let y = 15 in 
     let x = x * y in 
        let f = function (z) z * y + x in 
           let x = y in 
                f(10) + f(15)
~~~

Evaluate the program above under both static and dynamic scoping. Under dynamic scoping note that the program can be seen as equivalent to this program below:

~~~
  let y = 15 in 
     let x = x * y in 
        let f = function (z) z * y + x in 
           let x = y in 
                (10 * y +x ) + (15 * y + x)
~~~


Dynamic scoping is troublesome in many ways. It is hard for a programmer to reason about dynamic scoping since whoever implements a function f must be careful about the calling environment of the function which defines not just the formal arguments but also the other identifiers that the function's body may reference. 

Under dynamic scoping simple change of variable names can change the meaning of the function in unexpected ways.

~~~
  let y = 15 in 
     let x = x * y in 
        let f = function (z) z * y + x in 
           let xNew = y in  ## User changed name of x to xNew
                f(10) + f(15)
~~~


We will now explain how static scoping is implemented using _closures_.


## Evaluating Lettuce with Function Calls

We are now ready to start evaluating Lettuce with function calls.

Thus far, we have values $\mathbb{R}$ (for reals), $\mathbb{B}$ (for booleans), $\mathbf{error}$ (special error).
Now we will augment it with a new type of value for functions that will be termed a _closure_.


### Closure

A closure is defined as a combination of two things: 

- A function definition `function(x) expr` where `x` is the formal parameter to the call and
`expr` is the body of the function call.
- An environment $\sigma$ that will define all variables occuring freely in `expr` but not `x`.


We will write a closure as 

$ \text{Closure}( x, e, \sigma)$ that represents a closure 
involving the function definition

`function(x) e` with environment $\sigma$.

Let us see some examples of Closures.

### Example 1

Consider a function definition

~~~
function(y) x + y * y
~~~

`y` is the formal parameter but `x` is an identifier that is not a formal parameter of the function.
We say that it occurs freely in the definition of the function.

An example of a closure will be

$\text{Closure}( \texttt{y},\ \underset{\text{Func. Expr.}}{ \underbrace{\texttt{x + y * y }}}, \underset{\text{Env.}\ \sigma}{\underbrace{\{ x \mapsto 10 \}}}  ) $.


__Note:__ All that is missing is a value for the formal parameter `y`. Everything else is in place to evaluate the
function. The moment `y` is available, we will have everything needed to execute the function body `x + y * y`.


### Example 2

Consider the function definition

~~~
   function(x) function(y) x + y - 2 * z
~~~

`x` is the formal parameter of the function and the expression is `function (y) x + y - 2 * z `.
Note that `z` occurs freely in the expression. The identifier `y` is not free since it is bound
to the formal parameter of the inner function.

A closure needs to specify a value for `z`. 

$\text{Closure}(x,\ (\texttt{function(y) x + y - 2 * z}),\  \{ z \mapsto 100 \} )$


## Operational Semantics with Function Call

Recall that $\text{eval}(\texttt{e}, \sigma) = v$ states that evaluating the expression `e` under the
environment $\sigma$ yields the value $v$.

Now our values can be of the form:
- Real numbers: $\mathbb{R}$,
- Booleans: $\mathbb{B}$,
- Closures: $\mathbb{C}$ of the form $\text{Closure}(x, \texttt{e}, \sigma)$,
- Error $\mathbf{error}$.

We will write a rule for handling function definitions for static typing. 
Let us recall what is static typing? It captures the environment at the time 
a function is defined. This is exactly what the closure does.

$$ \begin{array}{c}
\\
\hline
\text{eval}(\texttt{FuncDef}(x, e), \sigma) = \text{Closure}(x, \texttt{e}, \sigma) \\
\end{array} \text{(func-def-ok)}$$

All it remains to specify what happens when we call a function.

This is the really important rule to understand: the rest will be easy if this particular idea sinks in. 
So pay very careful attention:

$$ \begin{array}{c}
\text{eval}(\texttt{fun_expr}, \sigma) = \text{Closure}(\color{blue}{p}, \color{red}{\texttt{e}}, \color{green}{\pi}),\ \text{eval}(\texttt{arg_expr}, \sigma) = v_2,\ v_2 \not= \mathbf{error} \\
\hline
\texttt{eval}( \texttt{FunCall(fun_expr, arg_expr)}, \sigma )  = \texttt{eval}( \color{red}{\texttt{e}}, \color{green}{\pi} [ \color{blue}{p} \mapsto v_2 ] )\\
\end{array} \text{(funcall-ok)}
$$


The rule concerns itself with evaluating a function call `FunCall(fun_expr, arg_expr)` of a call `fun_expr(arg_expr)` where
- `fun_expr` represents the function we are going to call.
- `arg_expr` represents the argument to the call.

First, if `fun_expr` is a function, then it better evaluate to a _closure_.
- This is expressed by the condition $\text{eval}(\texttt{fun_expr}, \sigma) = \text{Closure}(\color{blue}{p}, \color{red}{\texttt{e}}, \color{green}{\pi})$ on the top of the bar. This closure has a formal param $p$, 
body $\texttt{e}$ and the environment that was saved at function define time is $\pi$.

Next, we evaluate the argument to the funciton call `arg_expr`. Let this evaluate to $v_2$ where $v_2$ is not an __error__ value.

Finally, we are ready to call the closure. Note that with the arrival of $v_2$ from previous step, we have
everything needed to evaluate a function call. So we can extend the environment $\pi$ by binding 
$p$ the formal parameter to $v_2$. With that done, everything is ready to evaluate the body of the closure
$\texttt{e}$.

We need to add error rules. If the function expression is not a closure:
$$ \begin{array}{c}
\text{eval}(\texttt{fun_expr}, \sigma) \not \in \mathbb{C}\\
\hline
\texttt{eval}( \texttt{FunCall(fun_expr, arg_expr)}, \sigma )  = \mathbf{error}\\
\end{array} \text{(funcall-nok-1)}
$$

If the argument to the call leads to error:
$$ \begin{array}{c}
\text{eval}(\texttt{fun_expr}, \sigma) = \text{Closure}(\color{blue}{x}, \color{red}{\texttt{e}}, \color{green}{\pi}),\ \text{eval}(\texttt{arg_expr}, \sigma) = \mathbf{error} \\
\hline
\texttt{eval}( \texttt{FunCall(fun_expr, arg_expr)}, \sigma )  = \mathbf{error}\\
\end{array} \text{(funcall-nok-2)}
$$



In [6]:
/* 1. Define the values */
sealed trait Value 
case class NumValue(d: Double) extends Value
case class BoolValue(b: Boolean) extends Value
/* -- Let us add Closure to the set of values --*/
case class Closure(x: String, e: Expr, pi: Map[String, Value]) extends Value
case object ErrorValue extends Value


/*2. Operators on values */

def valueToNumber(v: Value): Double = v match {
    case NumValue(d) => d
    case _ => throw new IllegalArgumentException(s"Error: Asking me to convert Value: $v to a number")
}

def valueToBoolean(v: Value): Boolean = v match {
    case BoolValue(b) => b
    case _ => throw new IllegalArgumentException(s"Error: Asking me to convert Value: $v to a boolean")
}

def valueToClosure(v: Value): Closure = v match {
    case Closure(x, e, pi) => Closure(x, e, pi)
    case _ =>  throw new IllegalArgumentException(s"Error: Asking me to convert Value: $v to a closure")
}


defined trait Value
defined class NumValue
defined class BoolValue
defined class Closure
defined object ErrorValue
defined function valueToNumber
defined function valueToBoolean
defined function valueToClosure

In [7]:
def evalExpr(e: Expr, env: Map[String, Value]): Value =  {
    
    /* Method to deal with binary arithmetic operations */
    
    def applyArith2 (e1: Expr, e2: Expr) (fun: (Double , Double) => Double) = {
        val v1 = valueToNumber(evalExpr(e1, env))
        val v2 = valueToNumber(evalExpr(e2, env))
        val v3 = fun(v1, v2)
        NumValue(v3)
    }  /* -- We have deliberately curried the method --*/
    
    /* Helper method to deal with unary arithmetic */
    def applyArith1(e: Expr) (fun: Double => Double) = {
        val v = valueToNumber(evalExpr(e, env))
        val v1 = fun(v)
        NumValue(v1)
    }
    
    /* Helper method to deal with comparison operators */
    def applyComp(e1: Expr, e2: Expr) (fun: (Double, Double) => Boolean) = {
        val v1 = valueToNumber(evalExpr(e1, env))
        val v2 = valueToNumber(evalExpr(e2, env))
        val v3 = fun(v1, v2)
        BoolValue(v3)
    }
    
   
    e match {
        case Const(f) => NumValue(f)
        
        case Ident(x) => {
            if (env contains x) 
                env(x)
            else 
                throw new IllegalArgumentException(s"Undefined identifier $x")
        }
    
        case True => BoolValue(true)
    
        case False => BoolValue(false)
    
        case Plus(e1, e2) => applyArith2 (e1, e2) ( _ + _ )
    
        case Minus(e1, e2) => applyArith2(e1, e2) ( _ - _ )
    
        case Mult(e1, e2) =>  applyArith2(e1, e2) (_ * _)
    
        case Div(e1, e2) => applyArith2(e1, e2) {
            case (_, 0.0) => throw new IllegalArgumentException(s"Divide by zero error in divisor ${e2}")
            case (v1, v2) => v1/v2
        }
    
        case Log(e1) => applyArith1(e1) {
            case v if v > 0.0 => math.log(v)
            case v => throw new IllegalArgumentException(s"Log of a negative number ${e} evaluates to ${v}!")
        }
    
        case Exp(e1) => applyArith1(e1)(math.exp)
    
        case Sine(e1) => applyArith1(e1)(math.sin)
    
        case Cosine(e1) => applyArith1(e1)(math.cos)
    
        case Geq(e1, e2) => applyComp(e1, e2)(_ >= _)
    
        case Eq(e1, e2) => applyComp(e1, e2)(_ == _)
    
        case And(e1, e2) => { /* Short circuit eval of And */
            val v1 = evalExpr(e1, env)
            v1 match {
                case BoolValue(false) => BoolValue(false)
                case BoolValue(true) => {
                    val v2 = evalExpr(e2, env)
                    v2 match {
                        case BoolValue(_) => v2
                        case _ => throw new IllegalArgumentException(
                            s"And of a boolean and non-boolean expr: ${e2} which evaluated to ${v2}")       
                    }
                }
                case _ => throw new IllegalArgumentException(s"And of a non-boolean expr: ${e1} which evaluted to ${v1}")
            }
        }
    
        case Or(e1, e2) => { /* Short circuit eval of OR*/
            val v1 = evalExpr(e1, env)
            v1 match {
                case BoolValue(true) => BoolValue(true)
                case BoolValue(false) => {
                    val v2 = evalExpr(e2, env)
                    v2 match {
                        case BoolValue(_) => v2
                        case _ => throw new IllegalArgumentException(
                            s"Or of a boolean and non-boolean expr: ${e2} which evaluated to ${v2}")       
                    }
                }
                case _ => throw new IllegalArgumentException(s"Or of a non-boolean expr: ${e1} which evaluted to ${v1}")
            }
        }
    
        case Not(e1) => {
            val v = evalExpr(e1, env)
            v match {
                case BoolValue(b) => BoolValue(!b)
                case _ => throw new IllegalArgumentException(s"Not of a non-boolean expr: ${e} which evaluated to ${v}")
            }
        }
    
        case IfThenElse(e1, e2, e3) => {
            val v = evalExpr(e1, env)
            v match {
                case BoolValue(true) => evalExpr(e2, env)
                case BoolValue(false) => evalExpr(e3, env)
                case _ => throw new IllegalArgumentException(s"If-then-else condition expr: ${e1} is non-boolean -- evaluates to ${v}")
            }
        }
    
        case Let(x, e1, e2) => {
            val v1 = evalExpr(e1, env)  // eval e1
            val env2 = env + (x -> v1) // create a new extended env
            evalExpr(e2, env2) // eval e2 under that.
        }
    
        case FunDef(x, e) => {
            Closure(x, e, env) // Return a closure with the current enviroment.
        }
        
        case FunCall(e1, e2) => {
            val v1 = evalExpr(e1, env)
            val v2 = evalExpr(e2, env)
            // Since evaluating e2 did not fail with an exception,
            // if I reach this point in my execution, I know that
            // neither v1 nor v2 are error.
            v1 match {
                case Closure(x, closure_ex, closed_env) => {
                    // First extend closed_env by binding x to v2
                    val new_env = closed_env + ( x -> v2)
                    // Evaluate the body of the closure under the extended environment.
                    evalExpr(closure_ex, new_env)
                }
                case _ => throw new IllegalArgumentException(s"Function call error: expression $e1 does not evaluate to a closure")
            }
        }
    }
}

def evalProgram(p: Program) = {
    val m: Map[String, Value] = Map[String,Value]()
    p match { 
        case TopLevel(e) => evalExpr(e, m)
    }
}

defined function evalExpr
defined function evalProgram

In [8]:
val v1 = evalProgram(p1)

v1: Value = NumValue(100.0)

In [9]:
val v2 = evalProgram(p2)

v2: Value = NumValue(235.0)

In [10]:
val v3 = evalProgram(p3)

v3: Value = NumValue(3.4392347752010837)

In [11]:
val v4 = evalProgram(p4) // Will say that f is undefined identifier since we have let f = ... f ... in ... pattern

: 

In [18]:
/*--
let pfact = function (self) 
               function (n)
                  if (n == 0)
                  then 1
                  else 
                     n * ( self (self) (n-1) )
        --*/

val self = Ident("self")
val n = Ident("n")
val e1 = Mult(n, FunCall( FunCall(self, self), Minus(n, Const(1))))
val e2 = IfThenElse(Eq(n, Const(0)), Const(1), e1)
val e3 = FunDef("n", e2)
val e4 = FunDef("self", e3)
val e5 = Let("pfact", e4, FunCall(FunCall(Ident("pfact"), Ident("pfact")), Const(10)))

val prog = TopLevel(e5)

self: Ident = Ident("self")
n: Ident = Ident("n")
e1: Mult = Mult(Ident(n),FunCall(FunCall(Ident(self),Ident(self)),Minus(Ident(n),Const(1.0))))
e2: IfThenElse = IfThenElse(Eq(Ident(n),Const(0.0)),Const(1.0),Mult(Ident(n),FunCall(FunCall(Ident(self),Ident(self)),Minus(Ident(n),Const(1.0)))))
e3: FunDef = FunDef(n,IfThenElse(Eq(Ident(n),Const(0.0)),Const(1.0),Mult(Ident(n),FunCall(FunCall(Ident(self),Ident(self)),Minus(Ident(n),Const(1.0))))))
e4: FunDef = FunDef(self,FunDef(n,IfThenElse(Eq(Ident(n),Const(0.0)),Const(1.0),Mult(Ident(n),FunCall(FunCall(Ident(self),Ident(self)),Minus(Ident(n),Const(1.0)))))))
e5: Let = Let(pfact,FunDef(self,FunDef(n,IfThenElse(Eq(Ident(n),Const(0.0)),Const(1.0),Mult(Ident(n),FunCall(FunCall(Ident(self),Ident(self)),Minus(Ident(n),Const(1.0))))))),FunCall(FunCall(Ident(pfact),Ident(pfact)),Const(10.0)))
prog: TopLevel = TopLevel(Let(pfact,FunDef(self,FunDef(n,IfThenElse(Eq(Ident(n),Const(0.0)),Const(1.0),Mult(Ident(n),FunCall(FunCall(Ident(self),Ident(self

In [19]:
evalProgram(prog)

res18: Value = NumValue(3628800.0)